In [1]:
import pandas as pd

In [2]:
file_path = r"C:\Users\sally\OneDrive\바탕 화면\25_1\guangginGu_project\착한가격업소_주소전처리ver.csv"

df_good = pd.read_csv(file_path, encoding="utf-8")  # 'utf-8' 인코딩 사용

print(df_good.head()) 

   번호    업종명        업소명         주요품목     가격      업소 전화번호  \
0   1     한식  59년왕십리성수점  인삼왕갈비(300g)  15000  02-462-6775   
1   2    미용업      가연미용실           커트  12000          NaN   
2   3    미용업      갤러리헤어    미용료\n(파마)  30000  02-465-3861   
3   4     양식      겨울나그네          돈까스   5500  02-498-3650   
4   5  기타요식업        고은별           커피   3000  02-453-4933   

                              주소 주차여부 포장여부 배달여부 예약여부 남여화장실 구분여부 단체이용 가능여부  \
0    서울특별시 광진구 동일로4길 15 (자양동) 1층  NaN  NaN  NaN    O        NaN         O   
1  서울특별시 광진구 아차산로30길 39 (자양동) 1층  NaN  NaN  NaN  NaN        NaN       NaN   
2      서울특별시 광진구 동일로72길 46 (중곡동)  NaN  NaN  NaN  NaN        NaN       NaN   
3      서울특별시 광진구 능동로 147-1 (화양동)  NaN  NaN  NaN  NaN        NaN       NaN   
4             서울특별시 광진구 자양로 177   NaN  NaN  NaN  NaN        NaN       NaN   

  무선인터넷 제공여부 반려동물 동반여부  유아시설여부                 주소_추출  
0        NaN       NaN     NaN    서울특별시 광진구 동일로4길 15  
1        NaN       NaN     NaN  서울특별시 광진구 아차산로30길 39  
2        Na

In [3]:
#pip install selenium beautifulsoup4 pandas

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException  # <- 이 부분 추가!
from selenium.common.exceptions import InvalidSessionIdException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [5]:
chrome_driver_path = "chromedriver.exe"  # chromedriver 경로 지정
service = Service(chrome_driver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service)

In [6]:
def get_naver_map_menus(df_good, driver):
    try:
        if driver.session_id is None:
            print("🚀 새로운 웹드라이버 세션 생성 중...")
            driver = webdriver.Chrome()
    except AttributeError:
        print("🚀 새로운 웹드라이버 세션 생성 중...")
        driver = webdriver.Chrome()

    menu_data = {}

    for index, row in df_good.iterrows():
        store_name = row["업소명"]
        search_url = f"https://map.naver.com/v5/search/{store_name}"

        try:
            driver.get(search_url)
        except:
            print("⚠️ 웹드라이버 세션 오류 발생. 다시 실행합니다.")
            driver.quit()
            driver = webdriver.Chrome()
            driver.get(search_url)

        time.sleep(5)

        # 🔹 searchIframe 내부로 진입
        try:
            WebDriverWait(driver, 3).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe"))
            )
        except:
            print(f"{store_name}: searchIframe 로딩 실패. 계속 진행.")
            continue

        # 검색 리스트에서 첫 번째 가게 클릭
        try:
            store_list = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul'))
            )
            first_store = store_list.find_element(By.TAG_NAME, 'a')
            first_store.send_keys(Keys.ENTER)
        except:
            print(f"{store_name}: 가게 클릭 실패. 계속 진행.")
            continue

        driver.switch_to.default_content()

        # 🔹 entryIframe 내부로 이동
        try:
            WebDriverWait(driver, 3).until(
                EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe"))
            )
        except:
            print(f"{store_name}: entryIframe 로딩 실패. 계속 진행.")
            continue

        # 🔥 '메뉴' 버튼 클릭
        try:
            menu_button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'menu')]"))
            )
            menu_button.click()
            print(f"✅ {store_name}: 메뉴 버튼 클릭 완료")
        except:
            print(f"{store_name}: 메뉴 버튼을 찾지 못했습니다. 계속 진행.")
            continue

        # 🔹 메뉴 탭이 로딩될 때까지 대기
        time.sleep(3)

        # 🔹 메뉴 리스트 크롤링
        menu_data[store_name] = []
        
        try:
            menu_items = WebDriverWait(driver, 3).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.E2jtL"))
            )
            for item in menu_items:
                try:
                    menu_name = item.find_element(By.CSS_SELECTOR, "span.lPzHi").text
                    menu_price = item.find_element(By.CSS_SELECTOR, "div.GXS1X em").text
                    menu_data[store_name].append((menu_name, menu_price + "원"))  # 가격 뒤에 '원' 붙이기
                except:
                    continue
        except:
            print(f"{store_name}: 메뉴 데이터 없음")

    return menu_data

In [8]:
my_menu = get_naver_map_menus(df_good, driver)

✅ 59년왕십리성수점: 메뉴 버튼 클릭 완료
59년왕십리성수점: 메뉴 데이터 없음
가연미용실: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
갤러리헤어: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
겨울나그네: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
고은별: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 곰곰탕: 메뉴 버튼 클릭 완료
✅ 구름초밥: 메뉴 버튼 클릭 완료
그린세탁: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 금강숯불생고기: 메뉴 버튼 클릭 완료
✅ 금쪽삼겹: 메뉴 버튼 클릭 완료
김미라헤어: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 능라도분식: 메뉴 버튼 클릭 완료
✅ 다비다식당: 메뉴 버튼 클릭 완료
✅ 달디빈: 메뉴 버튼 클릭 완료
동현세탁: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
래빗컴퍼니(샵 고은별): 가게 클릭 실패. 계속 진행.
리오스포토스튜디오: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 리플커피 건국대점: 메뉴 버튼 클릭 완료
✅ 마니아우동: 메뉴 버튼 클릭 완료
✅ 마미가: 메뉴 버튼 클릭 완료
머리천국미용실: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
모짜르트헤어샵: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 무병장수한방삼계탕: 메뉴 버튼 클릭 완료
✅ 무한정수제돈가스: 메뉴 버튼 클릭 완료
미도미용실: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
방미헤어컬쳐: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
베이지크: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 보광식당: 메뉴 버튼 클릭 완료
보리&치킨: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 새맛식당: 메뉴 버튼 클릭 완료
서승미헤어: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 석기시대짜장마을: 메뉴 버튼 클릭 완료
세실리아미용실: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
세종원: 메뉴 버튼을 찾지 못했습니다. 계속 진행.
✅ 손수제치킨: 메뉴 버튼 클릭 완료
✅ 송영옥진주육전냉면: 메뉴 버튼 클릭 완료
✅ 스시붐: 메뉴 버튼 클릭 완료
양컷퍼드업헤

In [16]:
for each in my_menu:
    print(my_menu[each])

[]
[('곰곰탕', '8,500원'), ('1인 제육무침', '5,000원'), ('국수만두세트', '12,500원'), ('곰탕세트', '13,000원'), ('홍곰탕', '9,000원'), ('특 돼지곰탕', '11,500원'), ('고기 비빔국수', '9,000원'), ('만두 3알', '5,000원')]
[('(가성비갑)모듬 초밥 (10p)+미니우동+죽', '14,000원'), ('(베스트구성)구름 특 모듬 (12p)+미니우동+죽', '18,500원'), ('(커플1위)2인 세트 초밥20p+튀김 4개+미니우동2개', '37,000원'), ('수제 왕 새우튀김 4p', '10,900원'), ('(사장님원픽)연어 초밥 10p + 우동 +죽', '19,900원'), ('(가성비갑)모듬 초밥 (10p)+미니우동+죽', '14,000원'), ('(베스트구성)구름 특 모듬 (12p)+미니우동+죽', '18,500원'), ('(손많이가는)구름 스페셜 (14p)+미니우동+죽', '26,500원'), ('(커플1위)2인 세트 초밥20p+튀김 4개+미니우동2개', '37,000원'), ('(사장님원픽)연어 초밥 10p + 우동 +죽', '19,900원'), ('혼술 사시미 15p+미니 회무침+미니 우동(1인분)', '27,000원'), ('모듬활어회덮밥 + 우동', '11,900원'), ('수제 왕 새우튀김 4p', '10,900원'), ('치킨 가라아게 8p+샐러드(1인분)', '10,000원'), ('고로케 + 샐러드', '7,000원'), ('시메사바(고등어초밥)(2p)', '4,000원'), ('수제 계란 초밥 2p', '2,500원'), ('유부 초밥 2p', '2,000원'), ('소라 초밥 2p', '3,600원'), ('가리비갈릭 2p', '3,000원'), ('사이다', '2,000원'), ('콜라', '2,000원'), ('제로 콜라 뚱캔 55ml', '2,000원'), ('제로 사이다', '2,000원'), ('특수상권 배달비 추가(워커힐)', '1

In [ ]:
#driver.quit()